# 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뛰기

노래 가사 문장에는 연극 대본처럼 "인명:"형식의 문장이 없을 것 같아, 그런 문장을 건너뛰는 코드는 제외했다.

In [3]:
import re 
import numpy as np
import tensorflow as tf

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


특수문자, 공백을 제거하고 문장 시작에는 <start> 끝에는 <end>를 넣는다.

In [4]:
# 정제한 문장을 담을 곳
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    
    # 문장 정제하고, 정제한 문장을 corpus에 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:5]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>']

정제된 문장이 corpus에 담겨있음을 확인

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # tokenizer가 기억할 단어 개수, 단어장의 크기는 12,000이상으로 설정
        filters=' ', # 이미 문장을 정제했으니 filters가 필요없음
        oov_token="<unk>" # oov=out-of-vocabulary, 단어장에 없는 단어는 어떤 토큰으로 대체할것이냐
    )
    tokenizer.fit_on_texts(corpus) # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다

    tensor = tokenizer.texts_to_sequences(corpus) # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다

    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주는 padding 메소드 사용
    # maxlen의 디폴트값은 none으로, corpus 내에서 가장 긴 문장의 길이가 된다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15) #토큰 개수가 15개를 넘는 문장은 제외
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1014    3]
 [  37   15 9061 ...  878  644    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe3f2bfa940>


데이터 토큰화-자연어를 기계가 이해할 수 있는 벡터로 변환-가 되었음을 확인

In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


단어장에 단어별로 쪼개져 들어가있음을 확인

In [7]:
print(tensor[:3, :])

[[   2   50    5   91  297   64   57    9  970 6048    3    0    0    0
     0]
 [   2   17 2643  873    4    8   11 6049    6  330    3    0    0    0
     0]
 [   2   35    7   37   15  164  283   28  299    4   47    7   43    3
     0]]


설정한대로 시퀀스 길이는 15이고, 남는 공간은 0으로 채워져있음을(padding) 확인

In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor[:, :-1] # <end>가 아니라 <pad>를 갈라낼 가능성이 높다. 
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   64   57    9  970 6048    3    0    0    0]
[  50    5   91  297   64   57    9  970 6048    3    0    0    0    0]


마지막 토큰을 잘라내서 소스 문장을 생성하고, <start>를 잘라내서 타켓 문장을 생성했음을 확인

# 평가 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split
#총 데이터의 20%를 평가 데이터셋으로 사용
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

In [10]:
print('enc train: ', enc_train.shape)
print('dec train: ', dec_train.shape)

enc train:  (140788, 14)
dec train:  (140788, 14)


# 인공지능 만들기

In [11]:
# 모델 정의
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 16
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

embedding_size는 워드 벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기. 값이 커질수록 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란만을 야기할 수 있다.

LSTM레이어의 hidden_size(hidden state의 차원수)도 같은 맥락. 모델에 얼마나 많은 일꾼을 둘 것이냐? 역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 갈 뿐이다.

In [12]:
# 모델 학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, batch_size=256, epochs=10, validation_data=(enc_val, dec_val))

Epoch 1/10
550/550 [==============================] - 278s 468ms/step - loss: 3.9264 - val_loss: 3.5116
Epoch 2/10
550/550 [==============================] - 263s 478ms/step - loss: 3.3656 - val_loss: 3.2813
Epoch 3/10
550/550 [==============================] - 264s 480ms/step - loss: 3.1634 - val_loss: 3.1426
Epoch 4/10
550/550 [==============================] - 264s 481ms/step - loss: 3.0134 - val_loss: 3.0375
Epoch 5/10
550/550 [==============================] - 265s 481ms/step - loss: 2.8753 - val_loss: 2.9494
Epoch 6/10
550/550 [==============================] - 266s 483ms/step - loss: 2.7411 - val_loss: 2.8725
Epoch 7/10
550/550 [==============================] - 266s 484ms/step - loss: 2.6114 - val_loss: 2.8080
Epoch 8/10
550/550 [==============================] - 265s 482ms/step - loss: 2.4840 - val_loss: 2.7506
Epoch 9/10
550/550 [==============================] - 265s 482ms/step - loss: 2.3576 - val_loss: 2.7054
Epoch 10/10
550/550 [==============================] - 265s 482m

model.fit(x_train, y_train, batch_size, epoches, validation_data=(x_val, y_val))

batch_size: 한 번에 학습할 때 사용하는 데이터의 개수로, 클수록 여러 데이터를 기억하고 있어야 하므로 메모리가 커야 한다(대신 학습 시간이 빨라진다). 작을수록 학습은 꼼꼼하게 이루어지지만 학습 시간이 느려진다.

epoches: 학습 데이터 반복 횟수

In [13]:
# 입력 받은 값을 예측해서 문장을 생성하는 모델
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

# 회고

아무리 해도 validation loss를 2.2 이하로 낮추는 것은 무리였다...

한 번 코드를 실행하는 데 30분 이상이 걸려 파라미터값을 냅다 이렇게 저렇게 바꿔보기가 어려웠고, 결국 무엇을 어떻게 조정 하면 validation 값이 낮아지는지 감을 잡지 못한 것이 패인이다.

'embedding_size가 너무 커서 단어의 정도를 극도로 많이 나눈 탓인가?', 'LSTM레이어의 hidden_size가 적은가?'라는 생각에 기반하여 각각 배수로 줄리고, 늘리며 테스트 해봤는데 줄이고 늘릴 때마다 validatioin loss가 줄어들긴 했으나 2.2 이하로는 결국 내려가지 못했다. 테스트 시간을 줄이기 위해 batch_size도 조금씩 늘려봤는데 그래도 마지막 실행했을 때 기준 45분 정도가 걸렸다.

그렇게 나온 값이 주어진 i love에 you만 붙은 문장이라 꽤 허탈했다. 오류가 있는 출력은 아니지만... 코드 구동한 시간을 봐서라도 뭔가 좀 더 써줄 순 없겠니? 맥북을 붙잡고 애원해봐도 기계는 답이 없다.

최대한 미리미리 Explolation을 진행해야 한다, 세세한 변수와 함수의 역할을 잘 모르는 상태에서 이 값 저 값 바꾸며 코드를 돌려보는 방식엔 한계가 있다는 깨달음을 준 한 주였다.